## Espaces d'action et d'états

**Grille du jeu** :
* chaque pilier de la grille de jeu à une hauteur h = 4 cases.
* la grille de jeu contient *col* * *nb li* = 4 * 4 = 16 piliers.
* la grille de jeu contient *nb piliers* * h = 16 * 4 = 64 cases.

**Espace d'action** : *nb col* * *nb li* = 4 * 4 = 16

**Espace d'états** :
* chaque case peut contenir un pion noir, un pion blanc ou être vide tant qu'elle est au dessus de tous les autres pion de son pilier.
* si le pilier possède x pions, il y a donc 2^x configurations possibles. Si h = 4, alors chaque pilier possède 2^(h+1)-1 = 2^(5)-1 = 31 configurations possibles.
* considérant le nombre de piliers, la grille possède donc 31^16 configurations possibles.
* Pour connaitre l'espace d'état, il faut retirer à ce compte toutes les solutions incorrectes ou redondantes :
    - les solutions ne prenant en compte la symétrie du jeu,
    - les solutions considérant que le nombre de pièces de chaque couleur est arbitraire alors qu'il est égal,
    - les solutions considérant plusieurs lignes / colonnes / diagonales remplies entièrement de pions noirs ou de pions blancs.
* Même en retirant toutes ces solutions incorrectes ou redondates, l'espace d'état reste assez grand. Si l'on doit appliquer des solutions d'apprentissage par renforcement pour créer une IA, nous partirons plutôt sur des algorithmes utilisant des aproximateurs de fonction (réseaux de neurones) plutôt que des algorithmes tabulaires.


## Représentation du jeu

**Grille du jeu** : la grille est représentée par une liste de listes de listes. La première liste contient quatre listes correspondant aux couches verticales de la grille. Chacune de ces listes contient quatre liste correspondant aux colonnes de ces couches verticales. Chacune de ces listes contient *None* si l'emplacement de la colonnes est vide, *0* si l'emplacement contient le pion du joueur 0, *1* si l'emplacement contient le pion du joueur 1.

[[[None, None, None, None], [None, None, None, None], [None, None, None, None], [None, None, None, None]], [[None, None, None, None], [None, None, None, None], [None, None, None, None], [None, None, None, None]], [[None, None, None, None], [None, None, None, None], [None, None, None, None], [None, None, None, None]], [[0, None, None, None], [None, None, None, None], [None, None, None, None], [None, None, None, None]]]

**Mouvement** : un mouvement est représenté par un tuple contenant la cordonnée x, y, z du coup qui est joué.


## Solutions d'IA

### Algorithme min-max avec élagage alpha-beta pruning

Pour un jeu à deux joueurs, l'algorithme min-max permet de trouver le meilleur coup possible pour un joueur. Il explore tous les mouvements possibles de chaque joueur en alternance en utilisant une fonction d'évaluation. L'algorithme min-max propose une méthode brute force, qui peut être améliorée par des techniques d'optimisation telles que l'élagage alpha-beta. L'élagage alpha-beta permet en effet de réduire le nombre de nœuds évalués dans un arbre de recherche en élaguant les branches qui ne peuvent pas conduire à une solution optimale. 

Pour modifier la performance de l'algorithme, il est possible de modifier la profondeur maximum de la recherche dans l'arbre.

#### Fonction d'évaluation utilisée

La fonction d'évaluation que nous avons utilisés permet d'évaluer la qualité d'un état de jeu donné. Le principe est le suivant :

Pour chaque rangée, colonne et diagonale de la grille, on comptabilise le nombre de pions blancs, noirs et de cases vides présents dans une fenêtre glissante dont la taille correspond au nombre de pions a aligner pour gagner. Plus la fenêtre contient un ratio "pions blancs / case vide" élévé, plus on attribue un score positif à la fenêtre, et plus elle contient un ratio "pions noirs / case vide" élévé, plus on attribue un score négatif à la fenêtre. La distribution des scores suit une fonction exponentielle. Ces valeurs de score sont sommées progressivement jusqu'à ce que tout le jeu soit évalué. Le score final est donc une évaluation de l'état du jeu. Les blancs cherchent à maximiser ce score tandis que les noirs cherchent à le minimiser.

#### Limites

La fonction d'évaluation utilisée peut rapidement devenir coûteuse en ressource calculatoire, dépendament de la taille de la grille, mais aussi de la condition de victoire. En effet, plus le nombre de pions à alligner est inférieur à la longueur de la ligne à évaluer, plus cela fait de segments à calculer.

#### Pistes d'amélioration

On pourrait utiliser un ordre de recherche heuristique pour explorer en priorité les coups les plus prometteurs, ce qui peut permettre de trouver rapidement des solutions de bonne qualité.

Plusieurs autres heuristiques pourrait être utilisées et combiner pour évaluer la qualité d'un état donné :

* Menace : Donnez un score plus élevé aux positions qui menacent de former une rangée, une colonne ou une diagonale de pions.
* Mobilité : Donnez un score plus élevé aux positions qui permettent à un joueur de jouer plus de coups au tour suivant.
* Contrôle du centre : Donnez un score plus élevé aux positions qui contrôlent le centre du plateau.
* Éloignement : Donnez un score plus élevé aux positions qui sont plus éloignées des pions de l'adversaire.
* Points de force : Donnez un score plus élevé aux positions qui ont plus de pions placés dans des emplacements stratégiques qui peuvent former plusieurs alignements.


## NOTES

L'IA considère que l'adversaire joue comme lui (même profondeur de voups virtuels joués dans l'arbre), ce qui peut biaiser son comportement

L'IA a tendance à se suicider à la fin quand elle considère qu'elle va perdre. En gros vu qu'elle à calculé la victoire prochaine de l'adversaire, et qu'elle considère que celui-ci ne fait jamais d'erreurs, si elle fait durer la partie, l'adversaire va remplir encore plus la grille d'allignements de sa couleur. Or, sa fonction de score est basée sur le nombre d'alignements. Donc elle préfère mettre fin à la partie.

Certain alignement potentiel anti gravité valent autant que les alignements valable

Optimiser la fonction grid_quality_score